# Apscheduler

## 설치 :

```cmd
pip install apscheduler 

```

## 모듈 불러오기 :

```python
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.base import JobLookupError
```

## apscheduler의 세가지 방식:

    1.Cron - Cron 표현식으로 사용자가 정의한 방식으로 수행

    2.Date - 사용자가 원하는 특정한 날짜에 수행

    3.Interval - 사용자가 원하는 일정한 주기로 수행

In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.base import JobLookupError
import time


def job_date():
    print("I'm working date job...", "| [time] "
          , str(time.localtime().tm_hour) + ":"
          + str(time.localtime().tm_min) + ":"
          + str(time.localtime().tm_sec))
 
 
def job_cron():
    print("I'm working cron job...", "| [time] "
          , str(time.localtime().tm_hour) + ":"
          + str(time.localtime().tm_min) + ":"
          + str(time.localtime().tm_sec))

def job_interval():
    print("I'm working interval job...", "| [time] "
          , str(time.localtime().tm_hour) + ":"
          + str(time.localtime().tm_min) + ":"
          + str(time.localtime().tm_sec))
    
    
# BackgroundScheduler 를 사용하면 stat를 먼저 하고 add_job 을 이용해 수행할 것을 등록해줍니다.
sched = BackgroundScheduler()
sched.start()


# interval - 매 3조마다 실행
sched.add_job(job_interval, 'interval', seconds=3, id="test_2")
 
# cron 사용 - 매 5초마다 job 실행
#   : id 는 고유 수행번호로 겹치면 수행되지 않습니다.
#   만약 겹치면 다음의 에러 발생 => 'Job identifier (test_1) conflicts with an existing job'
sched.add_job(job_cron, 'cron', second='*/5', id="test_1")
 
# cron 으로 하는 경우는 다음과 같이 파라미터를 상황에 따라 여러개 넣어도 됩니다.
#   매시간 59분 10초에 실행한다는 의미.
sched.add_job(job_cron, 'cron', minute="59", second='10', id="test_10")

count = 0
while True:
    print("Running main process...............")
    time.sleep(1)

Running main process...............
Running main process...............
Running main process...............
Running main process...............I'm working interval job...
 | [time]  1:7:8
Running main process...............
I'm working cron job... | [time]  1:7:10
Running main process...............
Running main process...............
I'm working interval job... | [time]  1:7:11
Running main process...............
Running main process...............
I'm working interval job...Running main process............... | [time] 
 1:7:14
I'm working cron job... | [time]  1:7:15
Running main process...............
Running main process...............
Running main process...............
I'm working interval job... | [time]  1:7:17
Running main process...............
Running main process...............
I'm working cron job... | [time]  1:7:20
I'm working interval job... | [time]  1:7:20
Running main process...............
Running main process...............
Running main process...............
Runni

In [ ]:
from apscheduler.jobstores.base import JobLookupError
from apscheduler.schedulers.background import BackgroundScheduler
import time

class Scheduler(object):

    # 클래스 생성시 스케쥴러 데몬을 생성합니다.
    def __init__(self):
        self.sched = BackgroundScheduler()
        self.sched.start()
        self.job_id=''

    # 클래스가 종료될때, 모든 job들을 종료시켜줍니다.
    def __del__(self):
        self.shutdown()

    # 모든 job들을 종료시켜주는 함수입니다.
    def shutdown(self):
        self.sched.shutdown()

    # 특정 job을 종료시켜줍니다.
    def kill_scheduler(self, job_id):
        try:
            self.sched.remove_job(job_id)
        except JobLookupError as err:
            print("fail to stop scheduler: %s" % err)
            return

    def hello(self, type, job_id):
        print("%s scheduler process_id[%s] : %d" % (type, job_id, time.localtime().tm_sec))

    # 스케쥴러입니다. 스케쥴러가 실행되면서 hello를 실행시키는 쓰레드가 생성되어집니다.
    # 그리고 다음 함수는 type 인수 값에 따라 cron과 interval 형식으로 지정할 수 있습니다.
    # 인수값이 cron일 경우, 날짜, 요일, 시간, 분, 초 등의 형식으로 지정하여, 
    # 특정 시각에 실행되도록 합니다.(cron과 동일)
    # interval의 경우, 설정된 시간을 간격으로 일정하게 실행실행시킬 수 있습니다.
    def scheduler(self, type, job_id):
        print("%s Scheduler Start" % type)
        if type == 'interval':
            self.sched.add_job(self.hello, type, seconds=10, id=job_id, args=(type, job_id))
        elif type == 'cron':
            self.sched.add_job(self.hello, type, day_of_week='mon-fri', 
                                                 hour='0-23', second='*/2', 
                                                 id=job_id, args=(type, job_id))

scheduler = Scheduler()
# cron 스케쥴러를 실행시키며, job_id는 "1" 입니다.
scheduler.scheduler('cron', "1")

# interval 스케쥴러를 실행시키며, job_id는 "2" 입니다.
scheduler.scheduler('interval', "2")

count = 0
while True:
    print("Running main process...............")
    time.sleep(1)
    count += 1
    if count == 10:
        scheduler.kill_scheduler("1")
        print("######### kill cron schedule ##########")
    elif count == 30:
        scheduler.kill_scheduler("2")
        print("######## kill interval schedule ##########")

cron Scheduler Start
interval Scheduler Start
Running main process...............
cron scheduler process_id[1] : 38
Running main process...............
Running main process...............
cron scheduler process_id[1] : 40
Running main process...............
Running main process...............
cron scheduler process_id[1] : 42
Running main process...............
Running main process...............
cron scheduler process_id[1] : 44
Running main process...............
Running main process...............
cron scheduler process_id[1] : 46
Running main process...............
######### kill cron schedule ##########
Running main process...............
interval scheduler process_id[2] : 47
Running main process...............
Running main process...............
Running main process...............
Running main process...............
Running main process...............
Running main process...............
Running main process...............
Running main process...............
Running main process..

In [ ]:
from datetime import datetime
import time
import os

from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()
scheduler.add_job(tick, 'date', seconds=3)
scheduler.start()
print('Press Ctrl+{0} to exit'.format('Break' if os.name == 'nt' else 'C'))

try:
    # This is here to simulate application activity (which keeps the main thread alive).
    while True:
        print('%s' % datetime.now())
        time.sleep(60)
        
except (KeyboardInterrupt, SystemExit):
    # Not strictly necessary if daemonic mode is enabled but should be done if possible
    scheduler.shutdown()

Press Ctrl+Break to exit
Tick! The time is: 2018-09-17 01:04:40.170860
Tick! The time is: 2018-09-17 01:04:43.200129
Tick! The time is: 2018-09-17 01:04:46.186404
Tick! The time is: 2018-09-17 01:04:49.186694
Tick! The time is: 2018-09-17 01:04:52.170978
Tick! The time is: 2018-09-17 01:04:55.201242
Tick! The time is: 2018-09-17 01:04:58.185540
Tick! The time is: 2018-09-17 01:05:01.182873
Tick! The time is: 2018-09-17 01:05:04.170105


In [ ]:
from datetime import datetime

from apscheduler.schedulers.blocking import BlockingScheduler


sched = BlockingScheduler()

def my_job():
    print(datetime.now())

# The job will be executed on November 6th, 2009
sched.add_job(my_job, 'cron', day_of_week='mon-fri', hour=2, minute=4)

sched.start()

2018-09-17 02:04:00.008228


In [1]:
from apscheduler.jobstores.base import JobLookupError
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime

class Scheduler(object):

    # 클래스 생성시 스케쥴러 데몬을 생성합니다.
    def __init__(self, func):
        self.sched = BackgroundScheduler()
        self.func = func
        self.job_id=''

    # 클래스가 종료될때, 모든 job들을 종료시켜줍니다.
    def __del__(self):
        self.shutdown()
    
    def start(self):
        self.sched.start()
        
    def scheduler(self, job_id):
        self.sched.add_job(self.func, 'cron', id=job_id, day_of_week='mon-fri', hour=2, minute=21)
        self.sched.add_job(self.func, 'cron', id=job_id+"_2", day_of_week='mon-fri', hour=2, minute=22)
        
def my_job():
    f = open("C:/Users/folwo/Desktop"+str(datetime.now().date())+".txt", 'w')
    f.close()

if __name__=="__main__":
    scheduler = Scheduler(my_job)
    # cron 스케쥴러를 실행시키며, job_id는 "1" 입니다.
    scheduler.scheduler("1")
    scheduler.start()

In [3]:
i=2
while(i<10):
    print(i)
    j=i
    i+=3
print(j)

2
5
8
8
